# Get started using Python to submit Q# programs to the Azure Quantum service

> **⚠ WARNING**:
> This sample makes use of paid services on Azure Quantum. The cost of running this sample *with the provided parameters* on IonQ in a Pay-As-You-Go subscription is approximately \\$1-\\$2 USD (or the equivalent amount in your local currency). This quantity is only an approximate estimate and should not be used as a binding reference. The cost of the service might vary depending on your region, demand and other factors.

We start by importing the `qsharp` package, as this package provides interoperability between Q# and Python.

In [1]:
import qsharp
import qsharp.azure

Once we've imported the `qsharp` package, we can use the `%%qsharp` magic command to define a Q# operation that we'd like to run on hardware via Azure Quantum services.

In this notebook, we'll use the example of a quantum random number generator.

In [2]:
%%qsharp
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Measurement;

operation SampleRandomNumber(nQubits : Int) : Result[] {

    // We prepare a register of qubits in a uniform
    // superposition state, such that when we measure,
    // all bitstrings occur with equal probability.
    use register = Qubit[nQubits];

    // Set qubits in superposition.
    ApplyToEachA(H, register);

    // Measure all qubits and return.
    return ForEach(MResetZ, register);

}

We can verify that our program works before submitting it by using the local full-state simulator:

In [3]:
result = SampleRandomNumber.simulate(nQubits=3)
result

[1, 1, 1]

Next, we can use the `qsharp.azure.connect` function to choose a quantum workspace. Replace the `resource_id` and location below with the resource ID for your workspace, as found in the Azure Portal.

In [4]:
resource_id = ""
location = ""

In [5]:
qsharp.azure.connect(resourceId=resource_id, location=location)

Authenticated using Azure.Identity.AzureCliCredential


Connected to Azure Quantum workspace chgranad-0-15-2103-test5 in location eastus.


[{'id': 'honeywell.hqs-lt-s1', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'honeywell.hqs-lt-s1-apival', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'honeywell.hqs-lt-s2', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'honeywell.hqs-lt-s2-apival', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'honeywell.hqs-lt-s1-sim', 'current_availability': {}, 'average_queue_time': 513},
 {'id': 'ionq.qpu', 'current_availability': {}, 'average_queue_time': 1409},
 {'id': 'ionq.simulator', 'current_availability': {}, 'average_queue_time': 0}]

Next, we can choose what target we would like to submit our Q# program to.

In [6]:
qsharp.azure.target("ionq.qpu")

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.qpu


{'id': 'ionq.qpu', 'current_availability': {}, 'average_queue_time': 1409}

Finally, we can sumbit our Q# program to run on actual hardware.

In [7]:
job = qsharp.azure.submit(SampleRandomNumber, nQubits=3, shots=1000, jobName="Generate 3-bit random number")

Submitting SampleRandomNumber to target ionq.qpu...
Job successfully submitted for 1000 shots.
   Job name: Generate 3-bit random number
   Job ID: d1229348-e3d9-4cbc-abe1-0a710a66d0a3


To query the status of a job once it has been submitted, use the `qsharp.azure.status` function:

In [8]:
qsharp.azure.status(job.id)

{'id': 'd1229348-e3d9-4cbc-abe1-0a710a66d0a3', 'name': 'Generate 3-bit random number', 'status': 'Succeeded', 'uri': 'https://portal.azure.com/#@microsoft.onmicrosoft.com/resource/subscriptions/916dfd6d-030c-4bd9-b579-7bb6d1926e97/resourceGroups/qdk-validations/providers/Microsoft.Quantum/Workspaces/chgranad-0-15-2103-test5/job_management?microsoft_azure_quantum_jobid=d1229348-e3d9-4cbc-abe1-0a710a66d0a3', 'provider': 'ionq', 'target': 'ionq.qpu', 'creation_time': '2021-09-28T02:51:05.9887623+00:00', 'begin_execution_time': '2021-09-28T02:59:57.565+00:00', 'end_execution_time': '2021-09-28T03:00:03.806+00:00'}

In [9]:
qsharp.azure.output(job.id)

{'[0,0,0]': 0.125,
 '[1,0,0]': 0.097,
 '[0,1,0]': 0.155,
 '[1,1,0]': 0.148,
 '[0,0,1]': 0.115,
 '[1,0,1]': 0.078,
 '[0,1,1]': 0.165,
 '[1,1,1]': 0.117}